In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('final_data.csv')
df.head()

,Unnamed: 0,access,account_number,accounting_year,cdp_region,change_in_emissions,city,city_wide_emissions_inventory,country,gases_included,...,3,4,5,6,7,8,9,10,11,12
0,0,public,35859,2016-01-01 - 2016-12-31,North America,Decreased,Cleveland,Yes,United States of America,GPC gases,...,4.176901,5.452295,5.879344,6.456642,6.405718,6.106342,5.429181,3.995512,2.980663,2.071119
1,1,public,35857,2015-01-01 - 2015-12-31,North America,Stayed the same,Cincinnati,Yes,United States of America,GPC gases,...,4.176901,5.452295,5.879344,6.456642,6.405718,6.106342,5.429181,3.995512,2.980663,2.071119
2,2,public,50565,NaN,North America,NaN,Toledo,No,United States of America,NaN,...,4.176901,5.452295,5.879344,6.456642,6.405718,6.106342,5.429181,3.995512,2.980663,2.071119
3,3,public,43910,2017-01-01 - 2017-12-31,North America,Decreased,Columbus,Yes,United States of America,GPC gases,...,4.176901,5.452295,5.879344,6.456642,6.405718,6.106342,5.429181,3.995512,2.980663,2.071119
4,4,public,73301,NaN,North America,NaN,Gretna,Intending to undertake in future,United States of America,NaN,...,5.430430,6.019141,6.403310,6.400589,6.196881,6.355976,5.966169,5.754829,4.616877,3.638774


In [3]:
df = df.rename(columns={"1": "solar_hours_lookup_jan", "2": "solar_hours_lookup_feb",
                  "3": "solar_hours_lookup_mar", "4": "solar_hours_lookup_apr",
                  "5": "solar_hours_lookup_may", "6": "solar_hours_lookup_jun",
                  "7": "solar_hours_lookup_jul", "8": "solar_hours_lookup_aug",
                  "9": "solar_hours_lookup_sep", "10": "solar_hours_lookup_oct",
                  "11": "solar_hours_lookup_nov", "12": "solar_hours_lookup_dec"})

In [4]:
df.columns

Index(['Unnamed: 0', 'access', 'account_number', 'accounting_year',
       'cdp_region', 'change_in_emissions', 'city',
       'city_wide_emissions_inventory', 'country', 'gases_included',
       'geocoded_column', 'inventory_boundary', 'last_update', 'organization',
       'population', 'population_year', 'primary_protocol',
       'primary_protocol_comment', 'reason_for_change', 'reporting_authority',
       'total_basic_emissions_gpc', 'total_basic_emissions_gpc_1',
       'total_scope_1_emissions_metric_tonnes_co2e',
       'total_scope_2_emissions_metric_tonnes_co2e', 'total_scope_3_emissions',
       'year_reported_to_cdp', 'coordinates', 'location', 'address',
       'addr_list', 'state', 'State', 'Number of Customers',
       'Average Monthly Consumption (kWh)', 'Average Price (cents/kWh)',
       'Average Monthly Bill (Dollar and cents)',
       'Solar panel pricing (cost per watt $/W)', 'Emissions (lbs/MWh)',
       'solar_hours_lookup_jan', 'solar_hours_lookup_feb',
       '

In [5]:
df['cost_of_installment_for_solar_panel'] = df['Solar panel pricing (cost per watt $/W)'].apply(lambda x: x*5000)

In [22]:
df['electricity_cost_from_conventional_sources']  = df['Average Price (cents/kWh)'].apply(lambda x: x/100)

In [27]:
df['electricity_usage'] = df['Average Monthly Consumption (kWh)']

In [28]:
list_solar_lookup = ['solar_hours_lookup_jan', 'solar_hours_lookup_feb',
       'solar_hours_lookup_mar', 'solar_hours_lookup_apr',
       'solar_hours_lookup_may', 'solar_hours_lookup_jun',
       'solar_hours_lookup_jul', 'solar_hours_lookup_aug',
       'solar_hours_lookup_sep', 'solar_hours_lookup_oct',
       'solar_hours_lookup_nov', 'solar_hours_lookup_dec']

for months in list_solar_lookup:
    month = months.split('_')[-1]
    # TODO: update the number of days for each month
    df['elec_generated_from_solar_panel_in_'+month] = df[months].apply(lambda x: x*37*30)

In [29]:
list_elec_generated = ['elec_generated_from_solar_panel_in_jan',
       'elec_generated_from_solar_panel_in_feb',
       'elec_generated_from_solar_panel_in_mar',
       'elec_generated_from_solar_panel_in_apr',
       'elec_generated_from_solar_panel_in_may',
       'elec_generated_from_solar_panel_in_jun',
       'elec_generated_from_solar_panel_in_jul',
       'elec_generated_from_solar_panel_in_aug',
       'elec_generated_from_solar_panel_in_sep',
       'elec_generated_from_solar_panel_in_oct',
       'elec_generated_from_solar_panel_in_nov',
       'elec_generated_from_solar_panel_in_dec']

In [39]:
df['electricity_usage'] = df['electricity_usage'].apply(lambda x: x.replace(",", ""))
df['electricity_usage'] = df['electricity_usage'].astype(float)

In [40]:
for months in list_elec_generated:
    month = months.split('_')[-1]
    df['recurring_electricity_cost_'+month+'_with'] = df.apply(lambda x: x['electricity_cost_from_conventional_sources']*
                                                   (x['electricity_usage']-x[months]), axis=1)
    df['recurring_electricity_cost_'+month+'_without'] = df.apply(lambda x: x['electricity_cost_from_conventional_sources']*
                                                    x['electricity_usage'], axis=1)

In [41]:
pd.set_option('display.max_columns', None)
df.head()

,Unnamed: 0,access,account_number,accounting_year,cdp_region,change_in_emissions,city,city_wide_emissions_inventory,country,gases_included,geocoded_column,inventory_boundary,last_update,organization,population,population_year,primary_protocol,primary_protocol_comment,reason_for_change,reporting_authority,total_basic_emissions_gpc,total_basic_emissions_gpc_1,total_scope_1_emissions_metric_tonnes_co2e,total_scope_2_emissions_metric_tonnes_co2e,total_scope_3_emissions,year_reported_to_cdp,coordinates,location,address,addr_list,state,State,Number of Customers,Average Monthly Consumption (kWh),Average Price (cents/kWh),Average Monthly Bill (Dollar and cents),Solar panel pricing (cost per watt $/W),Emissions (lbs/MWh),solar_hours_lookup_jan,solar_hours_lookup_feb,solar_hours_lookup_mar,solar_hours_lookup_apr,solar_hours_lookup_may,solar_hours_lookup_jun,solar_hours_lookup_jul,solar_hours_lookup_aug,solar_hours_lookup_sep,solar_hours_lookup_oct,solar_hours_lookup_nov,solar_hours_lookup_dec,cost_of_installment_for_solar_panel,electricity_cost_from_conventional_sources,elec_generated_from_solar_panel_in_jan,elec_generated_from_solar_panel_in_feb,elec_generated_from_solar_panel_in_mar,elec_generated_from_solar_panel_in_apr,elec_generated_from_solar_panel_in_may,elec_generated_from_solar_panel_in_jun,elec_generated_from_solar_panel_in_jul,elec_generated_from_solar_panel_in_aug,elec_generated_from_solar_panel_in_sep,elec_generated_from_solar_panel_in_oct,elec_generated_from_solar_panel_in_nov,elec_generated_from_solar_panel_in_dec,total_electricity_cost_jan,total_electricity_cost_feb,total_electricity_cost_mar,total_electricity_cost_apr,total_electricity_cost_may,total_electricity_cost_jun,total_electricity_cost_jul,total_electricity_cost_aug,total_electricity_cost_sep,total_electricity_cost_oct,total_electricity_cost_nov,total_electricity_cost_dec,electricity_usage,recurring_electricity_cost_jan_with,recurring_electricity_cost_jan_without,recurring_electricity_cost_feb_with,recurring_electricity_cost_feb_without,recurring_electricity_cost_mar_with,recurring_electricity_cost_mar_without,recurring_electricity_cost_apr_with,recurring_electricity_cost_apr_without,recurring_electricity_cost_may_with,recurring_electricity_cost_may_without,recurring_electricity_cost_jun_with,recurring_electricity_cost_jun_without,recurring_electricity_cost_jul_with,recurring_electricity_cost_jul_without,recurring_electricity_cost_aug_with,recurring_electricity_cost_aug_without,recurring_electricity_cost_sep_with,recurring_electricity_cost_sep_without,recurring_electricity_cost_oct_with,recurring_electricity_cost_oct_without,recurring_electricity_cost_nov_with,recurring_electricity_cost_nov_without,recurring_electricity_cost_dec_with,recurring_electricity_cost_dec_without
0,0,public,35859,2016-01-01 - 2016-12-31,North America,Decreased,Cleveland,Yes,United States of America,GPC gases,"{'type': 'Point', 'coordinates': [-81.6944, 41...",A metropolitan area,2020-05-01T03:57:54.890,City of Cleveland,396815.0,2010,Global Protocol for Community-Scale Greenhouse...,NaN,Technological change,"CDP Cities, ICLEI - Local Governments for Sust...",7846700,1254024,8929029.0,3076598.0,536417.0,2018,"[41.4993, -81.6944]","Public Square, Superior Avenue East, East 4th ...","Public Square, Superior Avenue East, East 4th ...","['Public Square', 'Superior Avenue East', 'Eas...",Ohio,Ohio,"4,964,855",914,12.56,114.80,3.04,"1,361",2.343837,3.17699,4.176901,5.452295,5.879344,6.456642,6.405718,6.106342,5.429181,3.995512,2.980663,2.071119,15200.0,0.1256,2601.659360,3526.459200,4636.360488,6052.047257,6526.071296,7166.872787,7110.346813,6778.039441,6026.390491,4435.018594,3308.536263,2298.941903,47876.841561,59492.327557,73432.687729,91213.713553,97167.455474,105215.922211,104505.955974,100332.175380,90891.464561,70903.833544,56755.215457,44074.710303,914.0,-211.970016,114.7984,-328.124876,114.7984,-467.528477,114.7984,-645.338736,114.7984,-704.876155,114.7984,-785.360822,114.7984,-778.26

In [42]:
df.columns

Index(['Unnamed: 0', 'access', 'account_number', 'accounting_year',
       'cdp_region', 'change_in_emissions', 'city',
       'city_wide_emissions_inventory', 'country', 'gases_included',
       ...
       'recurring_electricity_cost_aug_with',
       'recurring_electricity_cost_aug_without',
       'recurring_electricity_cost_sep_with',
       'recurring_electricity_cost_sep_without',
       'recurring_electricity_cost_oct_with',
       'recurring_electricity_cost_oct_without',
       'recurring_electricity_cost_nov_with',
       'recurring_electricity_cost_nov_without',
       'recurring_electricity_cost_dec_with',
       'recurring_electricity_cost_dec_without'],
      dtype='object', length=101)

In [43]:
list_with = ['recurring_electricity_cost_jan_with',
       'recurring_electricity_cost_feb_with',
       'recurring_electricity_cost_mar_with',
       'recurring_electricity_cost_apr_with',
       'recurring_electricity_cost_may_with',
       'recurring_electricity_cost_jun_with',
       'recurring_electricity_cost_jul_with',
       'recurring_electricity_cost_aug_with',
       'recurring_electricity_cost_sep_with',
        'recurring_electricity_cost_oct_with',
        'recurring_electricity_cost_nov_with',
        'recurring_electricity_cost_dec_with']

list_without = ['recurring_electricity_cost_jan_without',
       'recurring_electricity_cost_feb_without',
       'recurring_electricity_cost_mar_without',
       'recurring_electricity_cost_apr_without',
       'recurring_electricity_cost_may_without',
       'recurring_electricity_cost_jun_without',
       'recurring_electricity_cost_jul_without',
       'recurring_electricity_cost_aug_without',
       'recurring_electricity_cost_sep_without',
        'recurring_electricity_cost_oct_without',
        'recurring_electricity_cost_nov_without',
        'recurring_electricity_cost_dec_without']

df['total_cost_with'] = df.loc[:,list_with].sum(axis=1)
df['total_cost_without'] = df.loc[:,list_without].sum(axis=1)

In [48]:
df.columns

Index(['Unnamed: 0', 'access', 'account_number', 'accounting_year',
       'cdp_region', 'change_in_emissions', 'city',
       'city_wide_emissions_inventory', 'country', 'gases_included',
       ...
       'recurring_electricity_cost_sep_with',
       'recurring_electricity_cost_sep_without',
       'recurring_electricity_cost_oct_with',
       'recurring_electricity_cost_oct_without',
       'recurring_electricity_cost_nov_with',
       'recurring_electricity_cost_nov_without',
       'recurring_electricity_cost_dec_with',
       'recurring_electricity_cost_dec_without', 'total_cost_with',
       'total_cost_without'],
      dtype='object', length=103)

In [49]:
df['breakeven'] = df['total_cost_without'] - df['total_cost_with']

In [55]:
df['profit'] = df['breakeven']*2 > df['cost_of_installment_for_solar_panel']

In [56]:
df['profit'].value_counts()

True     112
False     15
Name: profit, dtype: int64